In [1]:
import nltk
import json
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
import numpy as np
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
#loading json file
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenizing each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #adding documents in corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words
print (len(words), "unique lemmatized words", words)

450 documents
39 classes ['Contact', 'Deliver', 'Dell XPS', 'Laptop', 'MSI laptop', 'Redmi K20 Pro', 'Redmi Note 7 Pro', 'Redmi Note 8 Pro', 'Samung Galaxy A51', 'Samung Galaxy A71', 'Samung Galaxy M40', 'Samung smartphone', 'TV', 'Xiaomi smartphone', 'dell Inspiron 5570', 'dell Inspiron 7370', 'dell laptop', 'goodbye', 'greeting', 'hours', 'login', 'mi led 4a pro tv', 'mi tv', 'mi tv 4s', 'mi tv 4x', 'msi GE75', 'msi GF63', 'msi GT75 titan', 'name', 'options', 'payment', 'product', 'register', 'samsung fhd smart led tv', 'samsung smart uhd tv', 'samsung tv', 'samsung uhd tv', 'shop', 'smartphone']
182 unique lemmatized words [',', '.', '100000', '100k', '130000', '130k', '150000', '150k', '160000', '160k', '200000', '200k', '25000', '25k', '30000', '30k', '330k', '350000', '4', '40000', '40k', '4a', '4x', '50000', '50k', '5570', '7', '7370', '8', '80000', '80k', '90000', '90k', 'a', 'a51', 'a71', 'address', 'am', 'any', 'anyone', 'are', 'available', 'be', 'best', 'brand', 'buy', 'bye'

In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [8]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [11]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print("model created")


Epoch 1/200
450/450 [==============================] - 0s 871us/step - loss: 3.6392 - accuracy: 0.0444
Epoch 2/200
450/450 [==============================] - 0s 441us/step - loss: 3.4269 - accuracy: 0.0933
Epoch 3/200
450/450 [==============================] - 0s 479us/step - loss: 3.1637 - accuracy: 0.1600
Epoch 4/200
450/450 [==============================] - 0s 796us/step - loss: 2.7736 - accuracy: 0.2511
Epoch 5/200
450/450 [==============================] - 0s 863us/step - loss: 2.5648 - accuracy: 0.2600
Epoch 6/200
450/450 [==============================] - 0s 935us/step - loss: 2.1962 - accuracy: 0.3422
Epoch 7/200
450/450 [==============================] - 0s 969us/step - loss: 2.1637 - accuracy: 0.3467
Epoch 8/200
450/450 [==============================] - 0s 623us/step - loss: 1.9689 - accuracy: 0.4267
Epoch 9/200
450/450 [==============================] - 0s 610us/step - loss: 1.8682 - accuracy: 0.4311
Epoch 10/200
450/450 [==============================] - 0s 953us/step - 

450/450 [==============================] - 0s 725us/step - loss: 0.3562 - accuracy: 0.8800
Epoch 156/200
450/450 [==============================] - 0s 787us/step - loss: 0.3494 - accuracy: 0.8756
Epoch 157/200
450/450 [==============================] - 0s 771us/step - loss: 0.2937 - accuracy: 0.8889
Epoch 158/200
450/450 [==============================] - 0s 709us/step - loss: 0.3278 - accuracy: 0.8733
Epoch 159/200
450/450 [==============================] - 0s 787us/step - loss: 0.2838 - accuracy: 0.8867
Epoch 160/200
450/450 [==============================] - 0s 716us/step - loss: 0.3076 - accuracy: 0.9067
Epoch 161/200
450/450 [==============================] - 0s 765us/step - loss: 0.2956 - accuracy: 0.9000
Epoch 162/200
450/450 [==============================] - 0s 749us/step - loss: 0.2934 - accuracy: 0.9022
Epoch 163/200
450/450 [==============================] - 0s 770us/step - loss: 0.3161 - accuracy: 0.8800
Epoch 164/200
450/450 [==============================] - 0s 701us/ste